# Model Interpretation

At this point we have selected the SVM as our preferred model to do the predictions. We will now study its behaviour by analyzing misclassified articles. Hopefully this will give us some insights on the way the model is working.

In [1]:
import pickle
import pandas as pd
import numpy as np
import random
import zipfile

Let's load what we need:

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
local_zip = '/content/Pickles.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(".")
zip_ref.close()

In [6]:
# Dataframe
path_df = "/content/Pickles/df.pickle"
with open(path_df, 'rb') as data:
    df = pickle.load(data)
    
# X_train
path_X_train = "/content/Pickles/X_train.pickle"
with open(path_X_train, 'rb') as data:
    X_train = pickle.load(data)

# X_test
path_X_test = "/content/Pickles/X_test.pickle"
with open(path_X_test, 'rb') as data:
    X_test = pickle.load(data)

# y_train
path_y_train = "/content/Pickles/Y_train.pickle"
with open(path_y_train, 'rb') as data:
    y_train = pickle.load(data)

# y_test
path_y_test = "/content/Pickles/Y_train.pickle"
with open(path_y_test, 'rb') as data:
    y_test = pickle.load(data)

# features_train
path_features_train = "/content/Pickles/features_train.pickle"
with open(path_features_train, 'rb') as data:
    features_train = pickle.load(data)

# labels_train
path_labels_train = "/content/Pickles/labels_train.pickle"
with open(path_labels_train, 'rb') as data:
    labels_train = pickle.load(data)

# features_test
path_features_test = "/content/Pickles/features_test.pickle"
with open(path_features_test, 'rb') as data:
    features_test = pickle.load(data)

# labels_test
path_labels_test = "/content/Pickles/labels_test.pickle"
with open(path_labels_test, 'rb') as data:
    labels_test = pickle.load(data)
    
# MNB Model
path_model = "/content/drive/MyDrive/TVTTDPT/best_mnbc.pickle"
with open(path_model, 'rb') as data:
    mnb_model = pickle.load(data)
    
# Category mapping dictionary
category_codes = {
    'giai-tri': 1,
    'phap-luat': 2,
    'kinh-doanh': 3,
}

category_names = {
    0: 'giai-tri',
    1: 'phap-luat',
    2: 'kinh-doanh',
}

Let's get the predictions on the test set:

In [7]:
predictions = mnb_model.predict(features_test)

Now we'll create the Test Set dataframe with the actual and predicted categories:

In [8]:
# Indexes of the test set
index_X_test = X_test.index

# We get them from the original df
df_test = df.loc[index_X_test]

# Add the predictions
df_test['prediction'] = predictions

# Clean columns
df_test = df_test[['content', 'category', 'category_code', 'prediction']]

# Decode
df_test['category_predicted'] = df_test['prediction']
df_test = df_test.replace({'Category_Predicted':category_names})

# Clean columns again
df_test = df_test[['content', 'category', 'category_predicted']]

In [10]:
df_test.head()

,content,category,category_predicted
1237,\nMỹMinh tinh Angelina Jolie đưa con trai Knox...,giai-tri,1
437,\nThế giới cần 8.000 máy bay chở hàng trong 2 ...,kinh-doanh,3
51,\nKhách mua căn hộ Babylon Tower được tặng khu...,kinh-doanh,3
579,\nQuảng NamSau khi bắn một người chết và ba ng...,phap-luat,2
238,\nÔng Trần Huy Thanh Tùng – Trưởng ban kiểm to...,kinh-doanh,3


Let's get the misclassified articles:

In [11]:
condition = (df_test['category'] != df_test['category_predicted'])

df_misclassified = df_test[condition]

df_misclassified.head(3)

,content,category,category_predicted
1237,\nMỹMinh tinh Angelina Jolie đưa con trai Knox...,giai-tri,1
437,\nThế giới cần 8.000 máy bay chở hàng trong 2 ...,kinh-doanh,3
51,\nKhách mua căn hộ Babylon Tower được tặng khu...,kinh-doanh,3


Let's get a sample of 3 articles. We'll define a function to make this process faster:

In [12]:
def output_article(row_article):
    print('Actual Category: %s' %(row_article['category']))
    print('Predicted Category: %s' %(row_article['category_predicted']))
    print('-------------------------------------------')
    print('Text: ')
    print('%s' %(row_article['content']))

We'll get three random numbers from the indexes:

In [13]:
random.seed(8)
list_samples = random.sample(list(df_misclassified.index), 3)
list_samples

[821, 985, 1245]

First case:

In [16]:
output_article(df_misclassified.loc[list_samples[0]])

Actual Category: phap-luat
Predicted Category: 2
-------------------------------------------
Text: 

TP HCMVào khách sạn với Trần Văn Mạnh, 34 tuổi, sau thời gian tán tỉnh nhau trên mạng xã hội, người phụ nữ bị hắn quay clip khỏa thân rồi tống tiền.
Ngày 13/11, Mạnh, ngụ tỉnh Phú Thọ, bị TAND TP HCM tăng hình phạt từ 7 tháng lên một năm tù về tội Cưỡng đoạt tài sản.
Theo bản án sơ thẩm, chị Nguyễn ngụ tỉnh Vĩnh Phúc, đã có gia đình và đang sống ở TP HCM. Đầu năm 2017, mỗi lần về quê chị thường hẹn Mạnh đến khách sạn "tâm sự". Gã trai lén quay clip chị khỏa thân.
Đến năm 2018, Mạnh nhắn tin yêu cầu Nguyễn đưa 5 triệu đồng nếu không sẽ gửi clip "nóng" cho bạn bè, người thân của chị này. Sợ bị ảnh hưởng cuộc sống gia đình, người đàn bà chuyển khoản cho hắn.
Ngày 14/4, sau khi vào Sài Gòn sống, Mạnh nhắn tin mượn Nguyễn một triệu đồng. Chị này không đồng ý, hắn dọa sẽ tung clip lên mạng. Nạn nhân than đang hết tiền, đưa trước 300.000 đồng và hẹn hôm sau sẽ đưa số còn lại. Mạnh nói, nếu để 

Second case:

In [15]:
output_article(df_misclassified.loc[list_samples[1]])

Actual Category: phap-luat
Predicted Category: 2
-------------------------------------------
Text: 

Nhật BảnSự cô đơn và buồn chán có thể làm gia tăng số vụ trộm cắp do người cao tuổi thực hiện tại Nhật Bản.
Trong khi số vụ bắt giữ tại Nhật ổn định theo hướng giảm dần trong vòng 17 năm qua, "tội phạm tóc bạc", hiện tượng người cao tuổi (trên 65 tuổi) phạm tội, lại có xu hướng khác biệt rõ rệt, theo số liệu do Bộ Tư pháp công bố ngày 24/11 trong sách trắng hàng năm về thống kê và xu hướng tội phạm.
Cụ thể, tội danh trộm cắp tài sản chiếm chưa đầy một nửa tổng số vụ phạm tội được ghi nhận, nhưng chiếm tới 70% số vụ bắt giữ người trên 70 tuổi. Trong gần 42.500 người cao tuổi bị bắt năm 2019, một phần ba là phụ nữ. Trung bình 10 phụ nữ bị bắt có 9 người trộm đồ trong siêu thị hoặc trộm cắp tài sản.
Shinichi Ishizuka, giáo sư luật kiêm giám đốc Trung tâm Nghiên cứu Tội phạm học thuộc Đại học Ryukoku, thành phố Kyoto, nhận định có một vài yếu tố đằng sau làn sóng "tội phạm tóc bạc".
Nhiều t

Third case:

In [17]:
output_article(df_misclassified.loc[list_samples[2]])

Actual Category: giai-tri
Predicted Category: 1
-------------------------------------------
Text: 

Thái Thị Hoa, thí sinh Việt Nam tại Miss Earth 2020, diện áo tắm cut-out thi Trang phục biển.
Trong phần thi cùng dàn người đẹp khu vực châu Á - Thái Bình Dương ngày 11/11, Thái Thị Hoa chọn đồ của nhà thiết kế Nguyễn Minh Tuấn - người từng phụ trách trang phục áo tắm cho Hoàng Hạnh năm ngoái. Bộ đồ màu xanh lục cùng các đường cut-out hai bên hông lấy cảm hứng từ chủ đề "Thiên thần sinh thái" của cuộc thi năm nay. Cô phối cùng áo choàng in hình các sinh vật biển.
Phần thi được ghi hình từ hồ bơi trên sân thượng một chung cư cao cấp ở TP HCM. Năm nay, Miss Earth tổ chức thi trực tuyến, thí sinh tự ghi hình và gửi về cho ban tổ chức trình chiếu, chấm điểm.
Thái Thị Hoa 26 tuổi, quê ở Gia Lai. Cô cao 1,75 mét, số đo ba vòng lần lượt 84 - 62 - 95 cm, từng thi Hoa hậu Hoàn vũ Việt Nam 2017, vào top 10 cuộc thi Global Asian Model World. Cô tốt nghiệp khoa Tài chính Ngân hàng, Đại học Sài Gòn, 